<a href="https://colab.research.google.com/github/ericyoc/simple-language-model-creation-demo/blob/main/simple_language_model_creation_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tabulate import tabulate

In [2]:
# Function to create input sequences
def create_input_sequences(texts, tokenizer, max_sequence_len):
    input_sequences = []
    for text in texts:
        token_list = tokenizer.texts_to_sequences([text])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
    return input_sequences

In [3]:
# Function to create the language model
def create_model(total_words, max_sequence_len, embedding_dim, lstm_units, dropout_rate):
    model = Sequential()
    model.add(Embedding(total_words, embedding_dim, input_length=max_sequence_len-1))
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    model.add(Bidirectional(LSTM(lstm_units)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(total_words, activation='softmax'))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [4]:
# Function to generate text based on a prompt
def generate_text(model, tokenizer, prompt, num_words, max_sequence_len):
    print("Prompt:", prompt)
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([prompt])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = tf.argmax(predicted, axis=-1).numpy()[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break
        prompt += " " + output_word
    return prompt

In [5]:
# Function to evaluate the model and print metrics
def evaluate_model(model, inputs, outputs):
    loss, accuracy = model.evaluate(inputs, outputs, verbose=0)
    perplexity = 2 ** loss
    metrics_table = [
        ['Metric', 'Value', 'Acceptable Range', 'Explanation'],
        ['Loss', loss, '< 1.0', 'Measure of how well the model predicts the next word'],
        ['Accuracy', accuracy, '> 0.8', 'Percentage of correct word predictions'],
        ['Perplexity', perplexity, '< 5.0', 'Measure of how well the model predicts the test data']
    ]
    print("Model Metrics:")
    print(tabulate(metrics_table, headers='firstrow', tablefmt='grid'))

In [6]:
# Main function
def main():
    # Sample dataset
    texts = [
        "The quick brown fox jumps over the lazy dog.",
        "The lazy dog sleeps all day long.",
        "The brown fox is very quick and agile.",
        "The dog and the fox are good friends.",
        "The quick brown fox and the lazy dog are characters in a popular sentence.",
        "The fox jumps over the lazy dog in the popular sentence.",
        "The quick brown fox is known for jumping over the lazy dog.",
        "In the popular phrase, the quick brown fox jumps over the lazy dog.",
        "The quick brown fox demonstrates agility by jumping over the lazy dog.",
        "The lazy dog doesn't mind the quick brown fox jumping over it.",
        "Jumping over the lazy dog is a characteristic of the quick brown fox.",
        "The popular sentence features the quick brown fox and the lazy dog.",
        "The quick brown fox jumps over the lazy dog, showcasing its speed.",
        "The lazy dog is a key element in the sentence about the quick brown fox.",
        "The sentence 'The quick brown fox jumps over the lazy dog' is a pangram."
    ]

    # Tokenize the text
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    total_words = len(tokenizer.word_index) + 1

    # Set model hyperparameters
    max_sequence_len = 20
    embedding_dim = 300
    lstm_units = 400
    dropout_rate = 0.4
    epochs = 1000

    # Create input sequences
    input_sequences = create_input_sequences(texts, tokenizer, max_sequence_len)

    # Create input and output sequences
    inputs, outputs = input_sequences[:, :-1], input_sequences[:, -1]
    outputs = tf.keras.utils.to_categorical(outputs, num_classes=total_words)

    # Create the language model
    model = create_model(total_words, max_sequence_len, embedding_dim, lstm_units, dropout_rate)

    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Custom training loop with early stopping
    batch_size = 32
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        model.fit(inputs, outputs, batch_size=batch_size, epochs=1, verbose=1, callbacks=[early_stopping])
        if early_stopping.stopped_epoch > 0:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    # Evaluate the model and print metrics
    evaluate_model(model, inputs, outputs)

    # Generate text based on a prompt
    prompt = "The quick brown"
    num_words = 10
    generated_text = generate_text(model, tokenizer, prompt, num_words, max_sequence_len)
    print("\nGenerated Text:")
    print(generated_text)
    print("Expected: The quick brown fox jumps over the lazy dog.")

    # Print token size and context window
    print("\nToken Size:", total_words)
    print("Context Window:", max_sequence_len - 1)

In [7]:
if __name__ == '__main__':
    main()

Epoch 1/1000
5/5 [==============================] - 8s 137ms/step - loss: 3.6793 - accuracy: 0.0573
Epoch 2/1000
5/5 [==============================] - 1s 96ms/step - loss: 3.4109 - accuracy: 0.0701
Epoch 3/1000
5/5 [==============================] - 0s 58ms/step - loss: 3.3205 - accuracy: 0.1019
Epoch 4/1000
5/5 [==============================] - 0s 99ms/step - loss: 3.2687 - accuracy: 0.0892
Epoch 5/1000
5/5 [==============================] - 0s 21ms/step - loss: 3.1791 - accuracy: 0.1401
Epoch 6/1000
5/5 [==============================] - 0s 21ms/step - loss: 3.0486 - accuracy: 0.1847
Epoch 7/1000
5/5 [==============================] - 0s 21ms/step - loss: 2.8648 - accuracy: 0.1847
Epoch 8/1000
5/5 [==============================] - 0s 21ms/step - loss: 2.6976 - accuracy: 0.2548
Epoch 9/1000
5/5 [==============================] - 0s 21ms/step - loss: 2.4985 - accuracy: 0.3057
Epoch 10/1000
5/5 [==============================] - 0s 23ms/step - loss: 2.2833 - accuracy: 0.3631
Epoch 11